In [1]:
import h5py
import time
import numpy as np
import torch as T
from Pyccat_v1 import CCAT_holo

DEVICE0=T.device('cpu')
measuredbeams='input_beams/GRASP_296GHz_101_101.txt'

In [2]:
model=CCAT_holo('CCAT_MODEL','output0')
#model.build_holo()
model.mk_FF_4maps(Device=DEVICE0)

In [3]:
model.holo_conf

{'Rx1': ([0, 0, 600], 'beam/on-axis.txt'),
 'Rx2': ([400, 400, 600], 'beam/400_400_600.txt'),
 'Rx3': ([400, -400, 600], 'beam/400_-400_600.txt'),
 'Rx4': ([-400, 400, 600], 'beam/-400_400_600.txt'),
 'Rx5': ([-400, -400, 600], 'beam/-400_-400_600.txt')}

In [4]:
S0=np.genfromtxt('input_beams/input_error_panel.txt')
#S0=np.zeros(5*146)
paraA=T.tensor([1.0,0,0,0,0,0]*4,dtype=T.float64).to(DEVICE0)
paraP=T.tensor(np.zeros(5*4),dtype=T.float64).to(DEVICE0)
start=time.time()
x=model.FF(T.tensor(S0,dtype=T.float64).to(DEVICE0),paraA,paraP)
print(time.time()-start) 

1.0933423042297363


In [6]:
Meas=T.tensor(np.genfromtxt(measuredbeams,delimiter=','),dtype=T.float64)
model.fit_LP(Meas,Device=DEVICE0,Map_N=4)
np.savetxt('OLD/output/inference/test/results_LP'+'.txt',model.result_LP.x)

0.5263687139405827
0.5135846287738328
0.5041518433249217
0.5033888230040533
0.5027289813957799
0.5026827987342034
0.502651265788704
0.5026420228188775
0.502629155464239
0.5026229869222905
0.5026213832304287
0.5026211383757322
0.502621044152292
0.5026209366743308
0.5026208738043525
0.502620840478578
0.5026208230925191
0.5026208131575989
0.502620809357888
0.5026208081960927
Cost time: 33.71845809999999


In [ ]:
[1e-2,1e-1,1e-2,1e-2,1e-2,5]

In [12]:
Meas=T.tensor(np.genfromtxt(measuredbeams,delimiter=','),dtype=T.float64)
model.fit_surface(Meas,Device=DEVICE0,constraint=[1,1,1,1,1,1,10],Map_N=4,Init_LP=model.result_LP.x)
np.savetxt('OLD/output/inference/test/results_test_LP'+'.txt',model.result.x)

0.0 0.0 0.0 0.0 0.0
0.0 0.5026208081960927
0.5026208081960927
3.5698802766936932e-06 0.009219476623177458 0.00022149870664747717 0.0005856661161843784 0.004001951588066224
0.0028497821168269218 4.433266909561113
4.450148854592292
4.265262315891055e-07 0.0011015351543700149 2.6464475370139434e-05 6.997488491685723e-05 0.00047814974108827644
4.0681395806437964e-05 0.2929351016962317
0.294652333874015
0.17486364456967146 0.048957384615643446 0.057420379498901206 0.03603893241024186 0.08240102644848261
0.0002083757020744775 0.4663258940163801
0.866215637261395
0.023548586967327735 0.005639919335103275 0.007709924683558737 0.004913928595960165 0.010683221638358071
5.568916464490146e-05 0.28516002628366055
0.33771129666861344
0.003956166181697581 0.0006471154153150074 0.001273366659394472 0.0008838328384953541 0.00139713053859245
4.3038121989029674e-05 0.2910025060267357
0.2992031557822196
0.0008351748897414083 0.0008623247552311244 0.0002480496879296223 0.0002418553155328785 0.0003402406976

In [ ]:
Meas=T.tensor(np.genfromtxt(measuredbeams,delimiter=','),dtype=T.float64)
model.fit_surface(Meas,Device=DEVICE0,constraint=[1e-2,1e-1,1e-2,1e-2,1e-2,1],Map_N=4,Init_LP=np.append(np.array([1,0,0,0,0,0]*4),np.array([0,0,0,0,0]*4)))
np.savetxt('OLD/output/inference/test/results_test'+'.txt',model.result.x)

In [ ]:
model.result.x

In [ ]:
(model.result.x[0:-44]-S0).max()*1000